In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar el conjunto de datos
data = pd.read_csv('../../Datos/SinProcesar/personality_dataset.csv')

# Mostrar las primeras filas del dataset
print("Primeras filas del dataset:")
print(data.head())

ModuleNotFoundError: No module named 'pandas'

In [ ]:
print("Información general del dataset:")
print(data.info())  # Información básica como tipos de datos y cantidad de valores no nulos


In [ ]:
print("\nResumen estadístico de las columnas numéricas:")
print(data.describe())  # Estadísticas descriptivas


In [ ]:
# 2. Verificar valores nulos en el dataset
print("\nCantidad de valores nulos por columna:")
print(data.isnull().sum())  # Sumar los valores nulos